In [1]:
import pandas as pd
import os

os.chdir('C:/Users/user654/PycharmProjects/NewsCrawling/2020_1~6')

print(os.listdir())

##### 데이터프레임 생성

In [2]:
df = pd.DataFrame()
for i in os.listdir():
    if 'Article' in i:
        monthCSV = pd.read_csv(i , engine = 'python', header = None)
        df = df.append(monthCSV, ignore_index = True)
    
df = df.rename(columns = {0 : 'date' , 1 : 'category', 2 : 'media', 3 : 'title', 4 : 'content', 5 : 'url'})
df.drop(['date', 'category', 'media','url'], axis = 1, inplace = True)

display(df.head(4), df.tail(4))

,title,content
0,아파트 공급↓…유동자금↑ 일상톡톡 플러스,올해 집값 좌우할 3가지 변수…풍부한 유동자금 기준금리 보유세 강화 매도 vs 매수...
1,폭스바겐 전기차 목표 상향에…국내 배터리업체 바쁜 2020년 전망,폭스바겐 전기차 2025년 100만대→150만대로 목표치 상향 헤럴드경제 이세진 기...
2,포토 홈플러스 57L 대용량 바구니 대여해드립니다,1일 서울 등촌동 홈플러스 강서점에서 모델들이 대용량 대여용 장바구니를 선보이고 있...
3,中인민은행 지준율 6일부터 0.5%P 인하,중국 정부도 신년을 맞아 경기 부양에 나섰다. 중국 중앙은행 인민은행 PBOC 은 ...


,title,content
355621,한국인디게임협회 사단법인 출범…최훈 협회장 선임,인디개발자의 권익과 상생협력 도모 서울 뉴시스 오동현 기자 한국인디게임의 발전과 업...
355622,큐렉소 인공관절 수술로봇 인도에 수출,머니투데이 김근희 기자 큐비스 조인트 첫 해외수출 큐비스 조인트 사진 큐렉소 의료로...
355623,‘中 불청객’ 괭생이모자반 656톤 밀려왔다…“韓 어민 직격탄”,중국서 유입돼 제주 등 남해 곳곳 퍼져 악취 풍기고 양식장·선박 운항 피해도 온난화...
355624,캠코 2조 들여 ‘자금난’ 기업 자산 사준다,코로나19 대응 유동성 지원 은성수 금융위원장 금융위 제공 정부가 2조원 규모의 기...


##### 데이콘 데이터~ 

In [3]:
train = pd.read_csv('C:/[dacon]fakeNews/news_train.csv')
test = pd.read_csv('C:/[dacon]fakeNews/news_test.csv')

train.drop(['n_id','date','ord'], axis =1, inplace = True)
test.drop(['n_id','date','ord'], axis =1, inplace = True)

train.tail(3)

,title,content,info
118742,"모나리자, 중원 지분 16.88% 양수 결정",똑똑해진 소비자..한국도 이젠 소형차 시대,1
118743,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,1
118744,"모나리자, 중원 지분 16.88% 양수 결정",2020년 한국 TV 2대중 1대 인터넷 연결된다,1


In [7]:
import numpy as np

sent = pd.concat([train['title'].drop_duplicates(), train['content'].drop_duplicates(), 
                train['title'].drop_duplicates(), train['title'].drop_duplicates()], ignore_index = True).tolist()

print(sent[0])
print(len(sent))

[마감]코스닥 기관 678억 순매도
57039


In [9]:
sent = np.array(list(set(sent)))

print(len(sent))

49770


##### ~데이콘 데이터

In [13]:
#sents = np.concatenate([df['content'].values, sent])


sents = list(df['content']) + list(sent)

len(sents)

405395

##### 전처리

In [14]:
import re
from eunjeon import Mecab
from tqdm.notebook import tqdm
from konlpy.tag import Hannanum
def text_preprocessing(text_list):
    
    token_list = []
    stopwords = ['습니다','됩니다','는다' , '으로', 'null', '에서', '으로' , '다고', 
                 '지만', '된다며', '전년', '라며', '라고', 'nbsp','된다','부터','보다',
                '합니다','세요']
    #tokenizer = Kkma()
    
    #tokenizer = Mecab()
    tokenizer = Hannanum()
    for index, text in enumerate(tqdm(text_list)):
        try:
            txt = re.sub('[^가-힣a-z新北韓日上中株美]', ' ', text.lower())
            #txt = re.sub('&nbsp')
            txt = txt.replace('&nbsp', ' ')
            txt = txt.strip()

            token = tokenizer.morphs(txt)
            token = [t for t in token if t not in stopwords]
            token = [t for t in token if (len(t) >=2) | (t in ['◆','▶','【','%' , '美', '미','러','북','北' , '韓', '日', '中', '上','新','株'])]
            token = [i.strip() for i in token]
            token_list.append(token)
        except:
            pass
        
    return token_list


procSent = text_preprocessing(sents)

In [15]:
len(procSent)

405395

In [16]:
import gensim

In [17]:
size_embedding = 200 #Dimensionality of the feature vectors
windows = 3 #Maximum distance between the current and predicted word within a sentence
min_count = 1 #Ignores words with total frequency lower than this


w2v_model = gensim.models.Word2Vec(sentences = procSent , 
                                   size = size_embedding,
                                   window = windows,
                                   min_count = min_count)

##### 모델 저장

In [ ]:
w2v_model.save("C:/[dacon]fakeNews/Hannanum_NaverNewsW2V.model")